In [5]:
!cat './data/tmp.cnf'

p cnf 15 38
1 2 3 4 5 0
6 7 8 9 10 0
11 12 13 14 15 0
-1 -2 0
-1 -3 0
-1 -4 0
-1 -5 0
-2 -3 0
-2 -4 0
-2 -5 0
-3 -4 0
-3 -5 0
-4 -5 0
-6 -7 0
-6 -8 0
-6 -9 0
-6 -10 0
-7 -8 0
-7 -9 0
-7 -10 0
-8 -9 0
-8 -10 0
-9 -10 0
-11 -12 0
-11 -13 0
-11 -14 0
-11 -15 0
-12 -13 0
-12 -14 0
-12 -15 0
-13 -14 0
-13 -15 0
-14 -15 0
-1 -11 0
-2 -12 0
-3 -13 0
-4 -14 0
-5 -15 0


In [6]:
import os
import pdb
import random
from collections import namedtuple

import scipy.sparse as sparse
import torch

from cnf import CNF
from util import DataSample, adj, adj_batch, init_edge_attr, to_sparse_tensor

Batch = namedtuple('Batch', ['x', 'adj', 'sol'])


def load_dir(path):
    data = []
    for filename in os.listdir(path):
        name, ext = os.path.splitext(filename)
        if ext != '.cnf':
            continue
        f = CNF.from_file(os.path.join(path, filename))
        if name.startswith('uu'):
            continue
        data.append(DataSample(filename, f, adj(f), None))
    return data

In [7]:
import numpy as np

In [8]:
data =load_dir('data')[0]

In [9]:

def adj_sign(n, m, occur):
    i = np.repeat(range(n), [len(lst) for lst in occur])
    j = np.concatenate(occur)
    v = np.ones(len(i), dtype=np.int64)
    return sparse.coo_matrix((v, (i, j)), shape=(n, m))


def adj(f):
    n, m, occur = f.n_variables, len(f.clauses), f.occur_list
    adj_pos = adj_sign(n, m, occur[1 : n + 1])
    adj_neg = adj_sign(n, m, occur[:n:-1])
    return (adj_pos, adj_neg)


def adj_batch(adjs, fstack):
    adjp, adjn = list(zip(*adjs))
    return fstack((sparse.block_diag(adjp), sparse.block_diag(adjn)))


def to_sparse_tensor(x):
    x = x.tocoo()
    i = torch.tensor(np.vstack((x.row, x.col)), dtype=torch.int64)
    v = torch.tensor(x.data, dtype=torch.float32)
    return torch.sparse_coo_tensor(i, v, torch.Size(x.shape))


def init_edge_attr(k):
    return torch.cat(
        (
            torch.tensor([1, 0], dtype=torch.float32).expand(k, 2),
            torch.tensor([0, 1], dtype=torch.float32).expand(k, 2),
        ),
        dim=0,
    )


def normalize(x):
    return 2 * x - 1


def unnormalize(x):
    return (x + 1) / 2


In [13]:
data.adj[1].toarray().shape

(15, 38)

<15x38 sparse matrix of type '<class 'numpy.int64'>'
	with 70 stored elements in COOrdinate format>

In [14]:
def init_tensors(sample, device):
    # [1,0,0] -> assigned False
    # [0,1,0] -> assigned True
    # [0,0,1] -> clause
    adj = sample.adj
    n, m = adj[0].shape[0], adj[0].shape[1]
    xv = torch.zeros(n, 3, dtype=torch.float32)
    sol = [x if random.random() < 0.5 else -x for x in range(n + 1)]
    xv[torch.arange(n), (torch.tensor(sol[1:]) > 0).long()] = 1
    xv = xv.to(device)
    xc = torch.tensor([0, 0, 1], dtype=torch.float32).repeat(m, 1).to(device)
    xev = init_edge_attr(n).to(device)
    xec = init_edge_attr(m).to(device)
    vadj = to_sparse_tensor(sparse.hstack(adj)).to(device)
    cadj = to_sparse_tensor(sparse.vstack(adj)).t().to(device)
    return Batch((xv, xc, xev, xec), (vadj, cadj), sol)


In [15]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [16]:
tensor = init_tensors(data, device)

In [35]:
tensor.x[1]

torch.Size([38, 3])

In [94]:
tensor.x[1].shape  #clauses

torch.Size([38, 3])

In [95]:
tensor.x[2].shape  #edge 1

torch.Size([30, 2])

In [96]:
tensor.x[3].shape  #edge 2

torch.Size([76, 2])

In [103]:
tensor.adj[0].shape

torch.Size([15, 76])

In [104]:
tensor.adj[1].shape

torch.Size([38, 30])